# Exercise 2.1 : Intrinsic Evaluation

## Acknowledgement

This notebook contains components of a notebook created by Pia Sommerauer. 
The original notebook (with more slides and examples, including code to create word embeddings) can be found here:

https://github.com/PiaSommerauer/distributional_semantics

## Introduction

This notebook walks you through Exercise 2.1 of Assignment 2. The goal of this exercise is to (1) get hands on experience in comparing word embeddings, and (2) carry out an intrinsic evaluation and reflect on it.

The notebook illustrates:

* how to load a distributional semantic model in Python (with indications of where to obtain such models)
* how to calculate distances between vectors
* how to create a ranking between vector pairs based on distances and compare this to a gold ranking

In [0]:
# We will be using gensim: https://pypi.org/project/gensim/
# It provides implementations of various forms of language modeling
# including functions to create and work with word embeddings

# You can install gensim by running `pip install gensim' on your commandline 
# You can run this from the notebook by uncommenting the line below:
# %pip install gensim

# Other packages you may want to install if you do not have them installed already
# %pip install pandas
# %pip install scipy


# You will only need to install each module once, so if you end up running this notebook multiple times,
# you'll want to skip this cell or comment the packages you have installed out again.



In [0]:
import gensim
# for loading a stored model 
from gensim.models import KeyedVectors

# pandas is a useful package for dealing with data structures
import pandas as pd


## Downloading a distributional semantic model

There are many high quality distributional semantic models available. They are created from large corpora and have large coverage. Creating such models requires a lot of data and computation, and the more data, the better the model.
You are therefore generally best of using one of these pretrained models.

Sommerauer's notebook walks you through creating your own models (https://github.com/PiaSommerauer/distributional_semantics)

For the exercises for this component, we will use existing models. 

***Note though: these models are big!***

The Google word embeddings created using word2vec can be found here (which we are using in the examples):

https://code.google.com/archive/p/word2vec/

Glove also has embeddings which take up a bit less space:

https://nlp.stanford.edu/projects/glove/

Note though, that you may have to apply a small conversion procedure before the gensim code works with glove embeddings (they are formatted slightly differently from the output of word2vec).

It is explained here: https://radimrehurek.com/gensim/scripts/glove2word2vec.html


In [8]:
#loading a stored model. 

# Please make sure that the path `../models/GoogleNews-vectors-negative300.bin.gz' points to the location where you stored your word embeddings 
# if you are using a non-binary model, you will need to change binary=True to binary=False
ds_model = KeyedVectors.load_word2vec_format('https://mynlpbucket2663523.s3-ap-southeast-1.amazonaws.com/GoogleNews-vectors-negative300.bin.gz', binary=True)

# a first test with the model (you can replace "student" by other words)
ds_model.most_similar("cricket")


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.4939422

In [0]:
# similarity: a small scale experiment. Feel free to play with this and replace the terms

cos_man_woman = ds_model.similarity('man', 'woman')
cos_man_dog = ds_model.similarity('man', 'dog')

cos_tree_plant = ds_model.similarity('tree','plant')
cos_plant_flower = ds_model.similarity('plant','flower')

print(f'Man and woman should be more similar than man and dog:')
if cos_man_woman > cos_man_dog:
    print('True!')
    print('man-woman', cos_man_woman)
    print('man-dog', cos_man_dog)
else:
    print('False')
    print('man-woman', cos_man_woman)
    print('man-dog', cos_man_dog)
if cos_tree_plant > cos_plant_flower:
    print('True!')
    print('tree-plant', cos_tree_plant)
    print('plant-flower', cos_plant_flower)
else: 
 
    print('False!')
    print('tree-plamt',cos_tree_plant)
    print('plant-flower',cos_plant_flower)

In [0]:
simlex_data = pd.read_csv('https://mynlpbucket2663523.s3-ap-southeast-1.amazonaws.com/SimLex-999.txt',sep='\t')

In [0]:
ds_scores = {}
human_scores = []
model_scores = []

for index, row in simlex_data.sort_values(by='SimLex999', ascending=False).iterrows():
    wordpair = row['word1'] + '-' + row['word2']
    human_scores.append(row['SimLex999'])
    ds_score = ds_model.similarity(row['word1'],row['word2'])
    model_scores.append(ds_score)
    ds_scores[wordpair] = ds_model.similarity(row['word1'],row['word2'])

    
### Also saving the ranked output by the model to a file for inspection
ds_ranked_output = open('ds_output_simlex_pairs.txt', 'w')
for index, word_pair in enumerate(sorted(ds_scores, key=ds_scores.get, reverse=True)):
    ds_ranked_output.write(str(index) + '\t' + word_pair + '\t' + str(ds_scores[word_pair]) + '\n')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
#calculate spearman rho

from scipy.stats import spearmanr

spearmanr(human_scores, model_scores)



SpearmanrResult(correlation=0.44196551091403796, pvalue=5.068221892023142e-49)